# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
#dependencies and setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#create dataframe of city weather from csv
city_weather_df = pd.read_csv("../output/citiesweather.csv", index_col = 0)

#preview frame
city_weather_df.head()

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Charters Towers,-20.1000,146.2667,76.05,69,100,7.20,AU,1623549336
1,Ushuaia,-54.8000,-68.3000,38.86,87,75,26.46,AR,1623549096
2,Kikwit,-5.0410,18.8162,69.26,47,16,3.29,CD,1623549337
3,Rossosh',51.1209,38.5116,57.29,96,81,5.97,RU,1623549337
4,Butaritari,3.0707,172.7902,82.47,73,23,6.55,KI,1623549337


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=gkey)

#customize the size of the figure
figure_layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

#define locations of interest and level of humidity
locations = city_weather_df[["Latitude", "Longitude"]]
humid = city_weather_df["Humidity"]

#build heatlayer
humidity_layer = gmaps.heatmap_layer(locations, weights = humid, 
                                 dissipating = True, max_intensity = 100,
                                 point_radius = 15)

#add heatlayer layer to map
fig.add_layer(humidity_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#create dataframe of cities that fit weather criteria
ideal_weather_df = city_weather_df.loc[(city_weather_df['Max Temperature'] > 70) & (city_weather_df['Max Temperature'] < 80) & (city_weather_df['Wind Speed'] < 10) & (city_weather_df['Cloudiness'] == 0),  :]

#view frame
ideal_weather_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
31,Cortez,37.3489,-108.5859,78.19,11,0,6.06,US,1623549345
211,Mashhad,36.2970,59.6062,75.34,43,0,4.61,IR,1623549656
333,Saint-Pierre,-21.3393,55.4781,71.24,73,0,6.91,RE,1623549885
451,Murgab,37.4966,61.9714,75.07,43,0,2.30,TM,1623550050
473,Qūchān,37.1060,58.5095,70.88,44,0,2.73,IR,1623550121
480,Zabul Province,32.2500,67.2500,72.63,18,0,6.93,AF,1623550123
508,San Juan,-31.5375,-68.5364,74.73,46,0,2.44,AR,1623550101


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
#store into new dataframe named hotel_df
hotel_df = ideal_weather_df

#create hotel name column within dataframe
hotel_df['Hotel Name'] = ''

#set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = { 
    "radius": 5000,
    "type": "lodging",
    "key": gkey,
}
#iterate through pandas dataframe
for index, row in hotel_df.iterrows():

#define city and location
    lat = row['Latitude']
    lng = row['Longitude']
    city = row['City']
    
#set location variable within call specifications
    target_coordinates = f'{lat}, {lng}'

#add location parameter
    params['location'] = target_coordinates

    print(f"Retrieving Results for {city}.")

#assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
#extract results
    results = response['results']
    
#add found hotel to dataframe
    try:
        print(f"Closest hotel to {city} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
#add 'none' to dataframe, if no hotel found or error
    except (KeyError, IndexError):
        hotel_df.loc[index, 'Hotel Name'] = "None"

        print("Missing field/result or no hotel within range.")
        
    print("------------")

ipykernel_launcher:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
Retrieving Results for Cortez.
Closest hotel to Cortez is Sand Canyon National 9 Inn.
------------
Retrieving Results for Mashhad.
C:\Users\12164\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
Closest hotel to Mashhad is Khanye Sabz Hotel.
------------
Retrieving Results for Saint-Pierre.
Closest hotel to Saint-Pierre is Lindsey Hôtel.
------------
Retr

In [6]:
#create new dataframe of only cities with hotels withing 5000m of coordinates
hotel_clean_df = hotel_df.loc[hotel_df['Hotel Name'] != 'None', :]
hotel_clean_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
31,Cortez,37.3489,-108.5859,78.19,11,0,6.06,US,1623549345,Sand Canyon National 9 Inn
211,Mashhad,36.2970,59.6062,75.34,43,0,4.61,IR,1623549656,Khanye Sabz Hotel
333,Saint-Pierre,-21.3393,55.4781,71.24,73,0,6.91,RE,1623549885,Lindsey Hôtel
473,Qūchān,37.1060,58.5095,70.88,44,0,2.73,IR,1623550121,گلیم دستبافت کیمیایی
508,San Juan,-31.5375,-68.5364,74.73,46,0,2.44,AR,1623550101,Hotel Provincial


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_clean_info = [info_box_template.format(**row) for index, row in hotel_clean_df.iterrows()]
locations = hotel_clean_df[["Latitude", "Longitude"]]

In [8]:
#build marker layer
markers = gmaps.marker_layer(locations, info_box_content= hotel_clean_info)

#add layer to figure with heatmap
fig.add_layer(markers)

#display
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…